In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：评估AutoML表格回归模型的BatchPrediction结果

<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
  <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
</a>

<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
  <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
  查看GitHub
</a>

<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
  <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
  在Vertex AI工作台中打开
</a>


## 概览

本笔记本演示了如何使用Vertex AI回归模型评估组件来评估AutoML回归模型。模型评估可以帮助您根据评估指标确定模型的性能，并在必要时改进模型。

### 目标

在本教程中，您将学习如何通过 Vertex AI 管道作业使用 `google_cloud_pipeline_components` 来评估 Vertex AI 模型资源：

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI `AutoML`
- Vertex AI `TabularDataset` (AutoML)
- Vertex AI `AutoMLTabularTrainingJob`
- Vertex AI `BatchPrediction`
- Vertex AI `Pipeline`
- Vertex AI `Model Registry`

执行的步骤包括：

- 创建一个 Vertex AI 数据集
- 配置一个 `AutoMLTabularTrainingJob`
- 运行 `AutoMLTabularTrainingJob`，以返回一个模型
- 导入一个预训练的 `AutoML 模型资源` 进入管道
- 运行一个 `batch prediction` 作业
- 使用 `regression evaluation component` 评估 AutoML 模型
- 将分类指标导入到 AutoML 模型资源中

### 数据集

此笔记本中使用的数据集是PetFinder数据集的一部分，可在Kaggle上的 [这里](https://www.kaggle.com/c/petfinder-adoption-prediction) 上找到。当前数据集仅是用于预测宠物年龄问题考虑的原始数据集的一部分。它包含以下字段：

- `Type`: 动物类型 (1 = 狗, 2 = 猫)
- `Age`: 列出时宠物的年龄，以月计
- `Breed1`: 宠物的主要品种
- `Gender`: 宠物的性别
- `Color1`: 宠物的颜色1 
- `Color2`: 宠物的颜色2
- `MaturitySize`: 成熟时的体型 (1 = 小型, 2 = 中型, 3 = 大型, 4 = 特大型, 0 = 未指定)
- `FurLength`: 毛发长度 (1 = 短毛, 2 = 中长毛, 3 = 长毛, 0 = 未指定)
- `Vaccinated`: 宠物是否接种了疫苗 (1 = 是, 2 = 否, 3 = 不确定)
- `Sterilized`: 宠物是否绝育（1 = 是, 2 = 否, 3 = 不确定）
- `Health`: 健康状况 (1 = 健康, 2 = 轻伤, 3 = 重伤, 0 = 未指定)
- `Fee`: 领养费用 (0 = 免费)
- `PhotoAmt`: 为此宠物上传的照片总数
- `Adopted`: 是否已被领养 (是/否)。

**备注**: 此数据集已移至公共云存储桶，并在本笔记本中从那里访问。

### 成本
本教程使用了谷歌云的收费组件：

* Vertex AI
* 云存储

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[云存储
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Colab或者Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行Jupyter笔记本

Google Cloud指南 [设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一组简化的指导：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，可以在终端窗口的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口的命令行中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下必要的软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install google-cloud-pipeline-components==1.0.17 {USER_FLAG} -q
! pip3 install --upgrade kfp google-cloud-pipeline-components {USER_FLAG} -q
! pip3 install --upgrade matplotlib {USER_FLAG} -q

### 重新启动内核

在安装额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在你开始之前

### 设置您的Google Cloud项目

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得$300的免费信用用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果您在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是支持Vertex AI的区域。建议您选择离您最近的区域。

- 美洲： `us-central1`
- 欧洲： `europe-west4`
- 亚太地区： `asia-east1`

您可能无法在训练Vertex AI时使用多区域存储桶。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

如果您正在进行现场教程会话，您可能会使用一个共享的测试帐户或项目。为了避免用户在创建的资源之间的名称冲突，您为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行身份验证，通过oAuth。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中键入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中键入"Storage对象管理员"，并选择**Storage对象管理员**。

5. 点击*创建*。将包含您的密钥的JSON文件下载到本地环境中。

6. 在下面的单元格中，将您的服务帐号密钥的路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建云存储桶

**无论您使用的是什么笔记本环境，都需要执行以下步骤。**

当您使用Cloud SDK运行Vertex AI管道作业时，您的作业会将管道工件存储到一个云存储桶中。在本教程中，您将创建一个Vertex AI管道作业，将评估指标和特征属性等工件保存到一个云存储桶中。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当你的存储桶尚不存在时：运行以下单元格以创建你的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

#### 服务账户

您可以使用服务账户来创建Vertex AI Pipeline作业。如果您不想使用项目的计算引擎服务账户，请将`SERVICE_ACCOUNT`设置为另一个服务账户ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账号访问权限

运行以下命令，为您的服务账号授予读取和写入管道工件的权限，这些工件存储在您在上一步创建的存储桶中。您只需要在每个服务账号上运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import kfp
import matplotlib.pyplot as plt
from google.cloud import aiplatform_v1
from kfp.v2 import compiler

### 初始化 Python 用于 Vertex AI SDK

为您的项目和相应的存储桶初始化Python用于Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建Vertex AI数据集

使用数据集来源在Vertex AI中创建一个受管理的表格数据集资源。

In [ ]:
DATA_SOURCE = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/petfinder-tabular-classification.csv"

In [ ]:
# Create the Vertex AI Dataset resource
dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset",
    gcs_source=DATA_SOURCE,
)

print("Resource name:", dataset.resource_name)

训练AutoML模型

使用创建的数据集训练一个简单的回归模型，使用`Age`作为目标列。

设置一个显示名称，并使用适当的数据类型为列转换创建`AutoMLTabularTrainingJob`。

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "[your-train-job-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is specified, use the default one
if (
    TRAINING_JOB_DISPLAY_NAME == ""
    or TRAINING_JOB_DISPLAY_NAME is None
    or TRAINING_JOB_DISPLAY_NAME == "[your-train-job-display-name]"
):
    TRAINING_JOB_DISPLAY_NAME = "train-pet-agefinder-automl_" + UUID

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练作业，具有以下参数：
- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：模型要生成的预测类型。例如：回归，分类
- `column_transformations`：应用于输入列（即目标列以外的列）的转换。每个转换可能会从列的值中产生多个结果值，并且所有这些值都用于训练。当为BigQuery Struct列创建转换时，应使用“.”作为分隔符拆分列。只有没有子项的列应该有转换。如果输入列没有任何转换，训练会忽略这样的列，除了目标列之外，目标列不应定义任何转换。只能传递`column_transformations`或`column_specs`中的一个。考虑使用`column_specs`，因为`column_transformations`最终将被弃用。如果没有传递`column_transformations`或`column_specs`中的任何一个，则使用本地凭据将尝试将`column_transformations`设置为“auto”。要执行此操作，本地凭据需要访问GCS或BigQuery训练数据源的读取权限。
- `optimization_objective`：要最小化或最大化的优化目标。
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

要了解更多关于`AutoMLTabularTrainingJob`类的信息，请点击[这里](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob)

In [ ]:
train_job = aiplatform.AutoMLTabularTrainingJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    optimization_prediction_type="regression",
    column_specs={
        "Type": "categorical",
        "Breed1": "categorical",
        "Gender": "categorical",
        "Color1": "categorical",
        "Color2": "categorical",
        "MaturitySize": "categorical",
        "FurLength": "categorical",
        "Vaccinated": "categorical",
        "Sterilized": "categorical",
        "Health": "categorical",
        "Fee": "numeric",
        "PhotoAmt": "numeric",
        "Adopted": "categorical",
    },
    optimization_objective="minimize-rmse",
)

print(train_job)

设置模型的显示名称。

In [ ]:
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [ ]:
# If no name is specified, use the default name
if (
    MODEL_DISPLAY_NAME == ""
    or MODEL_DISPLAY_NAME is None
    or MODEL_DISPLAY_NAME == "[your-model-display-name]"
):
    MODEL_DISPLAY_NAME = "pet-agefinder-prediction-model_" + UUID

接下来，通过调用`run`方法开始训练任务，并提供以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `target_column`：要预测模型值的列名。
- `training_fraction_split`：用于训练的数据集的百分比。
- `validation_fraction_split`：用于验证的数据集的百分比。
- `test_fraction_split`：用于测试（留存数据）的数据集的百分比。
- `model_display_name`：经过训练模型的可读名称。
- `disable_early_stopping`：如果为true，整个预算将被使用。
- `budget_milli_node_hours`：（可选）以millihours为单位指定的最大训练时间（1000 = 小时）。

培训任务大约需要1.5-2小时才能完成。

In [ ]:
# Run the training job
model = train_job.run(
    dataset=dataset,
    target_column="Age",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name=MODEL_DISPLAY_NAME,
    disable_early_stopping=False,
    budget_milli_node_hours=1000,
)

训练完成后，获取模型评估结果并打印出来。

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

In [ ]:
# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

创建评估管道

现在，您运行Vertex AI BatchPrediction作业，并生成对其结果的评估和特征归因。

为此，您可以使用[`google-cloud-pipeline-components`](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/index.html) Python软件包中提供的组件创建一个Vertex AI管道。

### 定义管道

在定义管道的流程时，首先获取模型资源。然后，从提供的源数据集中对批量预测进行采样并创建一个批量预测任务。在创建批量预测任务时启用解释功能以生成特征归因。一旦批量预测任务完成，您将从结果中获取回归评估指标和特征归因。

该管道使用以下组件：

- `GetVertexModelOp`：获取 Vertex 模型工件。更多详情，请查看[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.GetVertexModelOp)。
- `EvaluationDataSamplerOp`：随机对输入数据集进行下采样至指定大小，用于计算 AutoML Tables 和自定义模型的 Vertex XAI 特征归因。创建一个使用 Apache Beam 的 Dataflow 作业来对数据集进行下采样。更多详情，请查看[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.EvaluationDataSamplerOp)。
- `ModelBatchPredictOp`：创建一个 Google Cloud Vertex BatchPredictionJob 并等待其完成。更多详情，请查看[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.ModelBatchPredictOp)。
- `ModelEvaluationRegressionOp`：对训练模型的批量预测结果计算评估指标。创建一个使用 Apache Beam 和 TFMA 的 Dataflow 作业来计算评估指标。支持表格数据的回归。 更多详情，请查看[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.ModelEvaluationRegressionOp)。
- `ModelEvaluationFeatureAttributionOp`：对训练模型的批量解释结果计算特征归因。创建一个使用 Apache Beam 和 TFMA 的 Dataflow 作业来计算特征归因。 更多详情，请查看[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.ModelEvaluationFeatureAttributionOp)。
- `ModelImportEvaluationOp`：使用 ModelService.ImportModelEvaluation 将模型评估工件导入现有 Vertex 模型。 更多详情，请查看[这里](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.ModelImportEvaluationOp)。

## 模型评估

In [ ]:
@kfp.dsl.pipeline(
    name="vertex-evaluation-automl-tabular-regression-feature-attribution"
)
def evaluation_automl_tabular_feature_attribution_pipeline(
    project: str,
    location: str,
    root_dir: str,
    model_name: str,
    target_column_name: str,
    batch_predict_gcs_source_uris: list,
    batch_predict_instances_format: str,
    batch_predict_predictions_format: str = "jsonl",
    batch_predict_machine_type: str = "n1-standard-4",
    batch_predict_explanation_metadata: dict = {},
    batch_predict_explanation_parameters: dict = {},
    batch_predict_explanation_data_sample_size: int = 10000,
    dataflow_max_num_workers: int = 5,
    dataflow_use_public_ips: bool = True,
    encryption_spec_key_name: str = "",
):

    from google_cloud_pipeline_components.aiplatform import ModelBatchPredictOp
    from google_cloud_pipeline_components.experimental.evaluation import (
        EvaluationDataSamplerOp, GetVertexModelOp,
        ModelEvaluationFeatureAttributionOp, ModelEvaluationRegressionOp,
        ModelImportEvaluationOp)

    # Get the Vertex AI model resource
    get_model_task = GetVertexModelOp(model_resource_name=model_name)

    # Run Data-sampling task
    data_sampler_task = EvaluationDataSamplerOp(
        project=project,
        location=location,
        root_dir=root_dir,
        gcs_source_uris=batch_predict_gcs_source_uris,
        instances_format=batch_predict_instances_format,
        sample_size=batch_predict_explanation_data_sample_size,
    )

    # Run Batch Explanations
    batch_explain_task = ModelBatchPredictOp(
        project=project,
        location=location,
        model=get_model_task.outputs["model"],
        job_display_name="model-registry-batch-predict-evaluation",
        gcs_source_uris=data_sampler_task.outputs["gcs_output_directory"],
        instances_format=batch_predict_instances_format,
        predictions_format=batch_predict_predictions_format,
        gcs_destination_output_uri_prefix=root_dir,
        machine_type=batch_predict_machine_type,
        encryption_spec_key_name=encryption_spec_key_name,
        # Set the explanation parameters
        generate_explanation=True,
        explanation_parameters=batch_predict_explanation_parameters,
        explanation_metadata=batch_predict_explanation_metadata,
    )

    # Run evaluation based on prediction type and feature attribution component.
    # After, import the model evaluations to the Vertex model.
    eval_task = ModelEvaluationRegressionOp(
        project=project,
        location=location,
        root_dir=root_dir,
        ground_truth_column=target_column_name,
        predictions_gcs_source=batch_explain_task.outputs["gcs_output_directory"],
        predictions_format=batch_predict_predictions_format,
        dataflow_max_workers_num=dataflow_max_num_workers,
        dataflow_use_public_ips=dataflow_use_public_ips,
        encryption_spec_key_name=encryption_spec_key_name,
    )

    # Get Feature Attributions
    feature_attribution_task = ModelEvaluationFeatureAttributionOp(
        project=project,
        location=location,
        root_dir=root_dir,
        predictions_format="jsonl",
        predictions_gcs_source=batch_explain_task.outputs["gcs_output_directory"],
        dataflow_max_workers_num=dataflow_max_num_workers,
        dataflow_use_public_ips=dataflow_use_public_ips,
        encryption_spec_key_name=encryption_spec_key_name,
    )

    ModelImportEvaluationOp(
        regression_metrics=eval_task.outputs["evaluation_metrics"],
        feature_attributions=feature_attribution_task.outputs["feature_attributions"],
        model=get_model_task.outputs["model"],
        dataset_type=batch_predict_instances_format,
    )

### 编译管道

接下来，将管道编译成`tabular_regression_pipline.json`文件。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=evaluation_automl_tabular_feature_attribution_pipeline,
    package_path="tabular_regression_pipeline.json",
)

定义运行流水线的参数

指定运行流水线所需的参数。

为您的流水线设置一个显示名称。

In [ ]:
PIPELINE_DISPLAY_NAME = "[your-pipeline-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is set, use the default one
if (
    PIPELINE_DISPLAY_NAME == "[your-pipeline-display-name]"
    or PIPELINE_DISPLAY_NAME == ""
    or PIPELINE_DISPLAY_NAME is None
):
    PIPELINE_DISPLAY_NAME = "pet_agefinder_" + UUID

要将所需参数传递给流水线，您需要在下面定义以下参数：

- `project`: 项目 ID。
- `location`: 运行流水线的区域。
- `root_dir`: 用于保存暂存文件和工件的 GCS 目录。一个随机子目录会在此目录下创建，用于保存作业信息，以便在发生故障时恢复作业。
- `model_name`: 已训练的 AutoML Tabular 回归模型的资源名称。
- `target_column_name`: 用作回归目标的列的名称。
- `batch_predict_gcs_source_uris`: 批量预测输入实例的 Cloud Storage 存储桶 URI 列表。
- `batch_predict_instances_format`: 批量预测输入实例的格式。可以是 "jsonl", "csv" 或 "bigquery"。
- `batch_predict_explanation_data_sample_size`: 用于批量预测和评估的样本大小。

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/pet_agefinder_{UUID}"
parameters = {
    "project": PROJECT_ID,
    "location": REGION,
    "root_dir": PIPELINE_ROOT,
    "model_name": model.resource_name,
    "target_column_name": "Age",
    "batch_predict_gcs_source_uris": [DATA_SOURCE],
    "batch_predict_instances_format": "csv",
    "batch_predict_explanation_data_sample_size": 3000,
}

接下来，您可以创建流水线作业，带有以下参数：

- `display_name`：此流水线的用户定义名称。
- `template_path`：PipelineJob或PipelineSpec JSON或YAML文件的路径。它可以是本地路径，Google Cloud Storage URI（例如“gs://project.name”）或Artifact Registry URI（例如“https://us-central1-kfp.pkg.dev/proj/repo/pack/latest”）。
- `parameter_values`：从运行时参数名称到控制流水线运行的值的映射。
- `enable_caching`：是否为运行打开缓存。如果未设置此项，则默认为编译时设置，对于所有任务，默认为True，但用户可以为单个任务指定不同的缓存选项。如果设置了此项，则该设置将应用于流水线中的所有任务。覆盖编译时设置。

使用配置的“SERVICE_ACCOUNT”运行流水线。

In [ ]:
job = aiplatform.PipelineJob(
    display_name=PIPELINE_DISPLAY_NAME,
    template_path="tabular_regression_pipeline.json",
    parameter_values=parameters,
    enable_caching=True,
)

job.run(service_account=SERVICE_ACCOUNT)

## 模型评估

在上一步的结果中，点击生成的链接查看在云控制台中的运行情况。

在用户界面中，当您点击这些流水线DAG节点时，许多节点会展开或折叠。以下是DAG的部分展开视图（点击图片查看大图）。

<img src="images/automl_tabular_regression_evaluation_pipeline.PNG" style="height:622px;width:726px"></img> -> <img src="images/automl_tabular_regression_evaluation_pipeline.PNG" style="height:622px;width:726px"></img>

获取模型评估结果

在评估流程完成后，请运行下面的单元格以打印评估指标。

In [ ]:
# Iterate over the pipeline tasks
for task in job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[
            0
        ]  # ['artifacts']
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### 可视化指标

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    if (
        i[0] == "meanAbsolutePercentageError"
    ):  # we are not considering MAPE as it is infinite. MAPE is infinite if groud truth is 0 as in our case Age is 0 for some instances.
        continue
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(10, 5))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

获取特征归因

特征归因表示模型中每个特征对于每个给定实例的预测贡献程度。

要了解更多关于特征归因的信息，请点击[这里](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview#feature_attributions)

运行以下单元格以获取特征归因。

In [ ]:
# Iterate over the pipeline tasks
for task in job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the feature-attribution task
    if (task.task_name == "feature-attribution") and (
        task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
        or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
    ):
        feat_attrs = task.outputs.get("feature_attributions").artifacts[0]
        feat_attrs_gcs_uri = feat_attrs.uri

print(feat_attrs)
print(feat_attrs_gcs_uri)

从获取到的特征归因的云存储URI中，获取归因值。

In [ ]:
# Load the results
attributions = !gsutil cat $feat_attrs_gcs_uri

# Print the results obtained
attributions = json.loads(attributions[0])
print(attributions)

### 可视化特征归因

使用柱状图可视化每个特征的获得的归因。

In [ ]:
data = attributions["explanation"]["attributions"][0]["featureAttributions"]
features = []
attr_values = []
for key, value in data.items():
    features.append(key)
    attr_values.append(value)

plt.figure(figsize=(5, 3))
plt.bar(x=features, height=attr_values)
plt.title("Feature Attributions")
plt.xticks(rotation=90)
plt.ylabel("Attribution value")
plt.show()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

将`delete_bucket`设置为**True**，以删除在此笔记本中创建的Cloud存储桶。

In [ ]:
# Delete model resource
model.delete()

# Delete the dataset resource
dataset.delete()

# Delete the training job
train_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI